In [2]:
!pip install git+https://github.com/mobiusml/hqq.git
!pip install git+https://github.com/mobiusml/gemlite.git #to use the gemlite backend
!pip install bitblas #to use the bitblas backend

  Cloning https://github.com/mobiusml/hqq.git to /tmp/pip-req-build-t04v4qo7
  Running command git clone --filter=blob:none --quiet https://github.com/mobiusml/hqq.git /tmp/pip-req-build-t04v4qo7
  Resolved https://github.com/mobiusml/hqq.git to commit b18f54fe9e39c81a5417b8a43cb3112f854442ba
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import torch
device        = 'cpu'
backend       = 'bitblas' #'torchao_int4' #"torchao_int4" (4-bit only) or "bitblas" (4-bit + 2-bit) or "gemlite" (8-bit, 4-bit, 2-bit, 1-bit)
compute_dtype = torch.bfloat16 if backend=="torchao_int4" else torch.float16
cache_dir     = None
model_id      = 'mobiuslabsgmbh/Llama-3.2-3B-Instruct_4bitgs64_hqq_hf'

is_prequantized = 'hqq_hf' in model_id
########################################################################
#Load model
from transformers import AutoModelForCausalLM, AutoTokenizer, HqqConfig

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=compute_dtype,
    cache_dir=cache_dir,
    device_map=device,
    low_cpu_mem_usage=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


RuntimeError: No GPU found. A GPU is needed for quantization.

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, HqqConfig

model = AutoModelForCausalLM.from_pretrained(
    model_id,
)

tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)

RuntimeError: No GPU found. A GPU is needed for quantization.

In [3]:
from hqq.utils.generation_hf import patch_model_for_compiled_runtime
model_id      = 'mobiuslabsgmbh/Llama-3.2-3B-Instruct_4bitgs64_hqq_hf'
patch_model_for_compiled_runtime(model_id, tokenizer)

prompt  = "Write an essay about large language models."
inputs  = tokenizer.apply_chat_template([{"role":"user", "content":prompt}], tokenize=True, add_generation_prompt=True, return_tensors="pt", return_dict=True)
outputs = model.generate(**inputs.to(model.device), max_new_tokens=1, cache_implementation="static", pad_token_id=tokenizer.pad_token_id)
#print(tokenizer.decode(outputs[0])

NameError: name 'tokenizer' is not defined

In [ ]:
model_id = 'mobiuslabsgmbh/Llama-2-70b-hf-2bit_g16_s128-HQQ'
import torch
import transformers
from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
model     = HQQModelForCausalLM.from_quantized(model_id, device_map='auto')


In [7]:
 from transformers import AutoModelForCausalLM, AutoTokenizer
 model_id = "mobiuslabsgmbh/Llama-3.2-3B-Instruct_4bitgs64_hqq_hf"
 tokenizer = AutoTokenizer.from_pretrained(model_id)
 model = AutoModelForCausalLM.from_pretrained(model_id)
 text = "What is AI?"
 inputs = tokenizer(text, return_tensors="pt")
 outputs = model.generate(**inputs, max_length=50)
 print(tokenizer.decode(outputs[0]))

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

RuntimeError: No GPU found. A GPU is needed for quantization.

In [8]:
from transformers import AutoModelForCausalLM, HqqConfig


model_id = "mobiuslabsgmbh/Llama-3.2-3B-Instruct_4bitgs64_hqq_hf"
# All linear layers will use the same quantization config
quant_config = HqqConfig(nbits=4, group_size=64)

# Load and quantize
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="cpu",
    quantization_config=quant_config
)

/usr/local/lib/python3.11/dist-packages/transformers/quantizers/auto.py:195: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


RuntimeError: No GPU found. A GPU is needed for quantization.

In [10]:
import torch
from transformers import AutoTokenizer
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import *
from hqq.utils.patching import *
from hqq.utils.generation_hf import HFGenerator

#Settings
###################################################
backend       = "bitblas" #bitblas or gemlite for 2-bit runtime
compute_dtype = torch.bfloat16 if backend=="torchao_int4" else torch.float16
device        = ''
cache_dir     = '.'

#Load the model
###################################################
model_id = 'mobiuslabsgmbh/Llama-3.2-3B-Instruct_4bitgs64_hqq_hf'
model     = AutoHQQHFModel.from_quantized(model_id, cache_dir=cache_dir, compute_dtype=compute_dtype, device=device, adapter='adapter_v0.1.lora').eval();
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)

#Use optimized inference kernels
###################################################
prepare_for_inference(model, backend=backend) #It takes a while...

#Generate
###################################################
#For longer context, make sure to allocate enough cache via the cache_size= parameter
#gen = HFGenerator(model, tokenizer, max_new_tokens=1000, do_sample=True, compile=None) #Slower generation but no warm-up
gen = HFGenerator(model, tokenizer, max_new_tokens=10, do_sample=True, compile="partial").warmup() #Faster generation, but warm-up takes a while

gen.generate("Write an essay about large language models", print_tokens=True)
#gen.generate("Tell me a funny joke!", print_tokens=True)
#gen.generate("How to make a yummy chocolate cake?", print_tokens=True)


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.37G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.57k [00:00<?, ?B/s]

Exception: Weight file missing. Check your cache directory.

In [1]:
import torch
from transformers import AutoTokenizer
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import *
from hqq.utils.patching import *
from hqq.utils.generation_hf import HFGenerator

# Settings
###################################################
backend       = "bitblas" #bitblas or gemlite for 2-bit runtime
compute_dtype = torch.bfloat16 if backend=="torchao_int4" else torch.float16
cache_dir     = '.'

# Device Selection: Auto-detect GPU or use CPU
if torch.cuda.is_available():
    device = 'cuda:0'
    print("GPU is available. Using CUDA.")
else:
    device = 'cpu'
    print("GPU is not available. Using CPU.")

# Load the model
###################################################
model_id = 'mobiuslabsgmbh/Llama-3.2-3B-Instruct_4bitgs64_hqq_hf'

try:
    model     = AutoHQQHFModel.from_quantized(model_id, cache_dir=cache_dir, compute_dtype=compute_dtype, device=device, adapter='adapter_v0.1.lora').eval()
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
except Exception as e:
    print(f"Error loading model on {device}: {e}")
    if device == 'cuda:0':
        print("Trying to load model on CPU instead due to potential GPU memory issues.")
        device = 'cpu' # Fallback to CPU if GPU loading fails
        try:
            model     = AutoHQQHFModel.from_quantized(model_id, cache_dir=cache_dir, compute_dtype=compute_dtype, device=device, adapter='adapter_v0.1.lora').eval()
            tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
        except Exception as e_cpu:
            print(f"Error loading model even on CPU: {e_cpu}")
            print("It seems there might be issues loading the model even on CPU. Please check your environment, disk space, and internet connection. ")
            exit() # Exit if loading fails even on CPU
    else: # If already tried CPU and failed
        print("Model loading failed on CPU. Please check your environment and model files.")
        exit()


# Use optimized inference kernels
###################################################
try:
    prepare_for_inference(model, backend=backend) #It takes a while...
    print(f"Model prepared for inference with backend '{backend}' on {device}.")
except Exception as e:
    print(f"Error preparing model for inference: {e}")
    print("Inference might still work, but performance could be suboptimal.")


# Generate
###################################################
#For longer context, make sure to allocate enough cache via the cache_size= parameter
#gen = HFGenerator(model, tokenizer, max_new_tokens=1000, do_sample=True, compile=None) #Slower generation but no warm-up
gen = HFGenerator(model, tokenizer, max_new_tokens=10, do_sample=True, compile="partial").warmup() #Faster generation, but warm-up takes a while

print("\nGenerating text:")
try:
    gen.generate("Write an essay about large language models", print_tokens=True)
    #gen.generate("Tell me a funny joke!", print_tokens=True)
    #gen.generate("How to make a yummy chocolate cake?", print_tokens=True)
except Exception as e:
    print(f"Error during text generation: {e}")
    print("Please check the error message and your environment.")

GPU is not available. Using CPU.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

Error loading model on cpu: Weight file missing. Check your cache directory.
Model loading failed on CPU. Please check your environment and model files.
Error preparing model for inference: name 'model' is not defined
Inference might still work, but performance could be suboptimal.


NameError: name 'model' is not defined

In [1]:
import torch
from transformers import AutoTokenizer
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import *
from hqq.utils.patching import *
from hqq.utils.generation_hf import HFGenerator

# Settings
###################################################
backend       = "bitblas" #bitblas or gemlite for 2-bit runtime
compute_dtype = torch.bfloat16 if backend=="torchao_int4" else torch.float16
cache_dir     = '.'

# Device Selection: Auto-detect GPU or use CPU
if torch.cuda.is_available():
    device = 'cuda:0'
    print("GPU is available. Using CUDA.")
else:
    device = 'cpu'
    print("GPU is not available. Using CPU.")

# Load the model
###################################################
model_id = 'mobiuslabsgmbh/Llama-3.2-3B-Instruct_4bitgs64_hqq_hf'

model = None  # Initialize model to None to avoid NameError if loading fails
tokenizer = None # Initialize tokenizer to None to avoid NameError if loading fails

try:
    model = AutoHQQHFModel.from_quantized(model_id, cache_dir=cache_dir, compute_dtype=compute_dtype, device=device, adapter='adapter_v0.1.lora').eval()
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
except Exception as e:
    print(f"Error loading model on {device}: {e}")
    if device == 'cuda:0':
        print("Trying to load model on CPU instead due to potential GPU memory issues.")
        device = 'cpu' # Fallback to CPU if GPU loading fails
        try:
            model = AutoHQQHFModel.from_quantized(model_id, cache_dir=cache_dir, compute_dtype=compute_dtype, device=device, adapter='adapter_v0.1.lora').eval()
            tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
        except Exception as e_cpu:
            print(f"Error loading model even on CPU: {e_cpu}")
            print("It seems there might be issues loading the model even on CPU. Please check your environment, disk space, and internet connection. ")
            exit() # Exit if loading fails even on CPU
    else: # If already tried CPU and failed
        print("Model loading failed on CPU. Please check your environment and model files.")
        exit()


# Use optimized inference kernels
###################################################
try:
    prepare_for_inference(model, backend=backend) #It takes a while...
    print(f"Model prepared for inference with backend '{backend}' on {device}.")
except Exception as e:
    print(f"Error preparing model for inference: {e}")
    print("Inference might still work, but performance could be suboptimal.")


# Generate
###################################################
#For longer context, make sure to allocate enough cache via the cache_size= parameter
#gen = HFGenerator(model, tokenizer, max_new_tokens=1000, do_sample=True, compile=None) #Slower generation but no warm-up
# Check if model and tokenizer are loaded before proceeding
if model is not None and tokenizer is not None:
    gen = HFGenerator(model, tokenizer, max_new_tokens=10, do_sample=True, compile="partial").warmup() #Faster generation, but warm-up takes a while

    print("\nGenerating text:")
    try:
        gen.generate("Write an essay about large language models", print_tokens=True)
        #gen.generate("Tell me a funny joke!", print_tokens=True)
        #gen.generate("How to make a yummy chocolate cake?", print_tokens=True)
    except Exception as e:
        print(f"Error during text generation: {e}")
        print("Please check the error message and your environment.")
else:
    print("Model or tokenizer loading failed. Please check the previous error messages.")

GPU is not available. Using CPU.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

Error loading model on cpu: Weight file missing. Check your cache directory.
Model loading failed on CPU. Please check your environment and model files.
Error preparing model for inference: 'NoneType' object has no attribute 'named_children'
Inference might still work, but performance could be suboptimal.
Model or tokenizer loading failed. Please check the previous error messages.


In [1]:
import os
import shutil
import huggingface_hub
from transformers import AutoTokenizer
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import *
from hqq.utils.patching import *
from hqq.utils.generation_hf import HFGenerator

# Settings
###################################################
backend = "bitblas"  # bitblas or gemlite for 2-bit runtime
compute_dtype = torch.bfloat16 if backend == "torchao_int4" else torch.float16
cache_dir = '.'

# Device Selection: Auto-detect GPU or use CPU
if torch.cuda.is_available():
    device = 'cuda:0'
    print("GPU is available. Using CUDA.")
else:
    device = 'cpu'
    print("GPU is not available. Using CPU.")

# Clear the Hugging Face cache
shutil.rmtree(os.path.expanduser("~/.cache/huggingface/transformers"), ignore_errors=True)

# Load the model
###################################################
model_id = 'mobiuslabsgmbh/Llama-3.2-3B-Instruct_4bitgs64_hqq_hf'

# Configure huggingface_hub to use the token:
huggingface_hub.hf_hub_download.HF_TOKEN = os.environ.get("HF_TOKEN")

model = None  # Initialize model to None to avoid NameError if loading fails
tokenizer = None  # Initialize tokenizer to None to avoid NameError if loading fails

try:
    model = AutoHQQHFModel.from_quantized(model_id, cache_dir=cache_dir, compute_dtype=compute_dtype, device=device,
                                        adapter='adapter_v0.1.lora').eval()
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
except Exception as e:
    print(f"Error loading model on {device}: {e}")
    if device == 'cuda:0':
        print("Trying to load model on CPU instead due to potential GPU memory issues.")
        device = 'cpu'  # Fallback to CPU if GPU loading fails
        try:
            model = AutoHQQHFModel.from_quantized(model_id, cache_dir=cache_dir, compute_dtype=compute_dtype,
                                                device=device, adapter='adapter_v0.1.lora').eval()
            tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
        except Exception as e_cpu:
            print(f"Error loading model even on CPU: {e_cpu}")
            print(
                "It seems there might be issues loading the model even on CPU. Please check your environment, disk space, and internet connection.")
            exit()  # Exit if loading fails even on CPU
    else:  # If already tried CPU and failed
        print("Model loading failed on CPU. Please check your environment and model files.")
        exit()

# Use optimized inference kernels
###################################################
try:
    prepare_for_inference(model, backend=backend)  # It takes a while...
    print(f"Model prepared for inference with backend '{backend}' on {device}.")
except Exception as e:
    print(f"Error preparing model for inference: {e}")
    print("Inference might still work, but performance could be suboptimal.")

# Generate
###################################################
# For longer context, make sure to allocate enough cache via the cache_size= parameter
# gen = HFGenerator(model, tokenizer, max_new_tokens=1000, do_sample=True, compile=None) #Slower generation but no warm-up
# Check if model and tokenizer are loaded before proceeding
if model is not None and tokenizer is not None:
    gen = HFGenerator(model, tokenizer, max_new_tokens=10, do_sample=True, compile="partial").warmup()  # Faster generation, but warm-up takes a while

    print("\nGenerating text:")
    try:
        gen.generate("Write an essay about large language models", print_tokens=True)
        # gen.generate("Tell me a funny joke!", print_tokens=True)
        # gen.generate("How to make a yummy chocolate cake?", print_tokens=True)
    except Exception as e:
        print(f"Error during text generation: {e}")
        print("Please check the error message and your environment.")
else:
    print("Model or tokenizer loading failed. Please check the previous error messages.")

GPU is not available. Using CPU.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

Error loading model on cpu: Weight file missing. Check your cache directory.
Model loading failed on CPU. Please check your environment and model files.
Error preparing model for inference: 'NoneType' object has no attribute 'named_children'
Inference might still work, but performance could be suboptimal.
Model or tokenizer loading failed. Please check the previous error messages.


In [8]:
import os
import shutil
import huggingface_hub
from transformers import AutoTokenizer
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import *
from hqq.utils.patching import *
from hqq.utils.generation_hf import HFGenerator
import transformers

# Settings
###################################################
backend = "bitblas"  # bitblas or gemlite for 2-bit runtime
compute_dtype = torch.bfloat16 if backend == "torchao_int4" else torch.float16
cache_dir = '.'

# Device Selection: Auto-detect GPU or use CPU
if torch.cuda.is_available():
    device = 'cuda:0'
    print("GPU is available. Using CUDA.")
else:
    device = 'cpu'
    print("GPU is not available. Using CPU.")

# Clear the Hugging Face cache
shutil.rmtree(os.path.expanduser("~/.cache/huggingface/transformers"), ignore_errors=True)

# Load the model
###################################################
model_id = 'mobiuslabsgmbh/Llama-3.2-3B-Instruct_4bitgs64_hqq_hf'

# Authenticate with Hugging Face Hub
#huggingface_hub.login()

# Reload Hugging Face Hub Configuration
#transformers.utils.hub.reload_module()

model = None  # Initialize model to None to avoid NameError if loading fails
tokenizer = None  # Initialize tokenizer to None to avoid NameError if loading fails

try:
    model = AutoHQQHFModel.from_quantized(model_id, cache_dir=cache_dir, compute_dtype=compute_dtype, device=device,
                                        adapter='adapter_v0.1.lora').eval()
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
except Exception as e:
    print(f"Error loading model on {device}: {e}")
    if device == 'cuda:0':
        print("Trying to load model on CPU instead due to potential GPU memory issues.")
        device = 'cpu'  # Fallback to CPU if GPU loading fails
        try:
            model = AutoHQQHFModel.from_quantized(model_id, cache_dir=cache_dir, compute_dtype=compute_dtype,
                                                device=device, adapter='adapter_v0.1.lora').eval()
            tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
        except Exception as e_cpu:
            print(f"Error loading model even on CPU: {e_cpu}")
            print(
                "It seems there might be issues loading the model even on CPU. Please check your environment, disk space, and internet connection.")
            exit()  # Exit if loading fails even on CPU
    else:  # If already tried CPU and failed
        print("Model loading failed on CPU. Please check your environment and model files.")
        exit()

# Use optimized inference kernels
###################################################
try:
    prepare_for_inference(model, backend=backend)  # It takes a while...
    print(f"Model prepared for inference with backend '{backend}' on {device}.")
except Exception as e:
    print(f"Error preparing model for inference: {e}")
    print("Inference might still work, but performance could be suboptimal.")

# Generate
###################################################
# For longer context, make sure to allocate enough cache via the cache_size= parameter
# gen = HFGenerator(model, tokenizer, max_new_tokens=1000, do_sample=True, compile=None) #Slower generation but no warm-up
# Check if model and tokenizer are loaded before proceeding
if model is not None and tokenizer is not None:
    gen = HFGenerator(model, tokenizer, max_new_tokens=10, do_sample=True, compile="partial").warmup()  # Faster generation, but warm-up takes a while

    print("\nGenerating text:")
    try:
        gen.generate("Write an essay about large language models", print_tokens=True)
        # gen.generate("Tell me a funny joke!", print_tokens=True)
        # gen.generate("How to make a yummy chocolate cake?", print_tokens=True)
    except Exception as e:
        print(f"Error during text generation: {e}")
        print("Please check the error message and your environment.")
else:
    print("Model or tokenizer loading failed. Please check the previous error messages.")

GPU is not available. Using CPU.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

Error loading model on cpu: Weight file missing. Check your cache directory.
Model loading failed on CPU. Please check your environment and model files.
Error preparing model for inference: 'NoneType' object has no attribute 'named_children'
Inference might still work, but performance could be suboptimal.
Model or tokenizer loading failed. Please check the previous error messages.


In [7]:
import os
if os.path.exists(os.path.expanduser("~/.huggingface/token")):
    print("Hugging Face token found. You are logged in.")
else:
    print("Hugging Face token not found. Login might have failed.")

Hugging Face token not found. Login might have failed.


In [ ]:
import os
if os.path.exists(os.path.expanduser("~/.huggingface/token")):
    print("Hugging Face token found. You are logged in.")
else:
    print("Hugging Face token not found. Login might have failed.")

In [1]:
import huggingface_hub
huggingface_hub.login()

In [2]:
import transformers
transformers.utils.hub.reload_module()

AttributeError: module 'transformers.utils.hub' has no attribute 'reload_module'

In [4]:
!huggingface-cli login --token hfْْْْْْْْْْْْْْْْْتانغخه

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `read` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `read`


In [1]:
import os
import shutil
import huggingface_hub
from transformers import AutoTokenizer
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import *
from hqq.utils.patching import *
from hqq.utils.generation_hf import HFGenerator
import transformers

# Settings
###################################################
backend = "bitblas"  # bitblas or gemlite for 2-bit runtime
compute_dtype = torch.bfloat16 if backend == "torchao_int4" else torch.float16
cache_dir = '.'

# Device Selection: Auto-detect GPU or use CPU
if torch.cuda.is_available():
    device = 'cuda:0'
    print("GPU is available. Using CUDA.")
else:
    device = 'cpu'
    print("GPU is not available. Using CPU.")

# Clear the Hugging Face cache
shutil.rmtree(os.path.expanduser("~/.cache/huggingface/transformers"), ignore_errors=True)

# Load the model
###################################################
model_id = 'mobiuslabsgmbh/Llama-3.2-3B-Instruct_4bitgs64_hqq_hf'

# Authenticate with Hugging Face Hub (Optional)
# huggingface_hub.login()

# Reload Hugging Face Hub Configuration (Optional)
# transformers.utils.hub.reload_module()

model = None  # Initialize model to None to avoid NameError if loading fails
tokenizer = None  # Initialize tokenizer to None to avoid NameError if loading fails

# Specify local file path (replace with your actual download path)
local_model_path = '/path/to/downloaded/model'

try:
    model = AutoHQQHFModel.from_pretrained(local_model_path, compute_dtype=compute_dtype, device=device,
                                          adapter='adapter_v0.1.lora').eval()  # Load from local path
    tokenizer = AutoTokenizer.from_pretrained(local_model_path)  # Load tokenizer from local path
except Exception as e:
    print(f"Error loading model: {e}")
    # ... (error handling as before) ...

# ... (rest of your code remains the same) ...

GPU is not available. Using CPU.
Error loading model: type object 'AutoHQQHFModel' has no attribute 'from_pretrained'


In [ ]:
import os
import shutil
import huggingface_hub
from transformers import AutoTokenizer
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import *
from hqq.utils.patching import *
from hqq.utils.generation_hf import HFGenerator
import transformers

# Settings
###################################################
backend = "bitblas"  # bitblas or gemlite for 2-bit runtime
compute_dtype = torch.bfloat16 if backend == "torchao_int4" else torch.float16
cache_dir = '.'

# Device Selection: CPU
device = 'cpu'
print("GPU is not available. Using CPU.")

# Clear the Hugging Face cache
shutil.rmtree(os.path.expanduser("~/.cache/huggingface/transformers"), ignore_errors=True)

# Load the model
###################################################
model_id = 'mobiuslabsgmbh/Llama-3.2-3B-Instruct_4bitgs64_hqq_hf'

# Authenticate with Hugging Face Hub (Optional)
huggingface_hub.login()

model = None  # Initialize model to None to avoid NameError if loading fails
tokenizer = None  # Initialize tokenizer to None to avoid NameError if loading fails

try:
    # Using from_quantized to load the pre-quantized model
    model = AutoHQQHFModel.from_quantized(model_id, cache_dir=cache_dir, compute_dtype=compute_dtype, device=device, adapter='adapter_v0.1.lora').eval()
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# ... (rest of your code remains the same) ...

In [1]:
import os
import shutil
import huggingface_hub
from transformers import AutoTokenizer
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import *
from hqq.utils.patching import *
from hqq.utils.generation_hf import HFGenerator
import transformers

# Settings
###################################################
backend = "bitblas"  # bitblas or gemlite for 2-bit runtime
compute_dtype = torch.bfloat16 if backend == "torchao_int4" else torch.float16
cache_dir = '.'

# Device Selection: CPU
device = 'cpu'
print("GPU is not available. Using CPU.")

# Clear the Hugging Face cache
shutil.rmtree(os.path.expanduser("~/.cache/huggingface/transformers"), ignore_errors=True)

# Load the model
###################################################
model_id = 'mobiuslabsgmbh/Llama-3.2-3B-Instruct_4bitgs64_hqq_hf'

# Authenticate with Hugging Face Hub (Optional)
#huggingface_hub.login()

model = None  # Initialize model to None to avoid NameError if loading fails
tokenizer = None  # Initialize tokenizer to None to avoid NameError if loading fails

try:
    # Using from_quantized to load the pre-quantized model
    model = AutoHQQHFModel.from_quantized(model_id, cache_dir=cache_dir, compute_dtype=compute_dtype, device=device, adapter='adapter_v0.1.lora').eval()
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# ... (rest of your code remains the same) ...

GPU is not available. Using CPU.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

Error loading model: Weight file missing. Check your cache directory.


In [ ]:
import os
import shutil
import huggingface_hub
from transformers import AutoTokenizer
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import *
from hqq.utils.patching import *
from hqq.utils.generation_hf import HFGenerator
import transformers

# Settings
###################################################
backend = "bitblas"  # bitblas or gemlite for 2-bit runtime
compute_dtype = torch.bfloat16 if backend == "torchao_int4" else torch.float16
cache_dir = '.'

# Device Selection: CPU
device = 'cpu'
print("GPU is not available. Using CPU.")

# Clear the Hugging Face cache
shutil.rmtree(os.path.expanduser("~/.cache/huggingface/transformers"), ignore_errors=True)

# Load the model
###################################################
model_id = 'mobiuslabsgmbh/Llama-3.2-3B-Instruct_4bitgs64_hqq_hf'

# Authenticate with Hugging Face Hub (Optional)
huggingface_hub.login()

model = None  # Initialize model to None to avoid NameError if loading fails
tokenizer = None  # Initialize tokenizer to None to avoid NameError if loading fails

try:
    # Using from_quantized to load the pre-quantized model
    model = AutoHQQHFModel.from_quantized(model_id, cache_dir=cache_dir, compute_dtype=compute_dtype, device=device, adapter='adapter_v0.1.lora').eval()
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# ... (rest of your code remains the same) ...

In [1]:
import os
import shutil
import huggingface_hub
from transformers import AutoTokenizer
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import *
from hqq.utils.patching import *
from hqq.utils.generation_hf import HFGenerator
import transformers

# Settings
###################################################
backend = "bitblas"  # bitblas or gemlite for 2-bit runtime
compute_dtype = torch.bfloat16 if backend == "torchao_int4" else torch.float16
cache_dir = '.'

# Device Selection: CPU
device = 'cpu'
print("GPU is not available. Using CPU.")

# Clear the Hugging Face cache
shutil.rmtree(os.path.expanduser("~/.cache/huggingface/transformers"), ignore_errors=True)

# Load the model
###################################################
model_id = 'mobiuslabsgmbh/Llama-3.2-3B-Instruct_4bitgs64_hqq_hf'

# Authenticate with Hugging Face Hub
#huggingface_hub.login()


model = None  # Initialize model to None to avoid NameError if loading fails
tokenizer = None  # Initialize tokenizer to None to avoid NameError if loading fails

try:
    # Using from_quantized to load the pre-quantized model
    model = AutoHQQHFModel.from_quantized(model_id, cache_dir=cache_dir, compute_dtype=compute_dtype, device=device, adapter='adapter_v0.1.lora').eval()
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
except Exception as e:
    print(f"Error loading model: {e}")
    # If initial loading fails, try manual download
    # (Replace /path/to/downloaded/model with actual path)
    local_model_path = '/path/to/downloaded/model'
    try:
        model = AutoHQQHFModel.from_pretrained(local_model_path, compute_dtype=compute_dtype, device=device, adapter='adapter_v0.1.lora').eval()
        tokenizer = AutoTokenizer.from_pretrained(local_model_path)
    except Exception as e2:
        print(f"Error loading model from local path: {e2}")
        exit()

# ... (rest of your code remains the same) ...

GPU is not available. Using CPU.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

Error loading model: Weight file missing. Check your cache directory.
Error loading model from local path: type object 'AutoHQQHFModel' has no attribute 'from_pretrained'


In [1]:
import os
import shutil
import huggingface_hub
from transformers import AutoTokenizer
from hqq.models.hf.base import AutoHQQHFModel
from hqq.core.quantize import *
from hqq.utils.patching import *
from hqq.utils.generation_hf import HFGenerator
import transformers

# Settings
###################################################
backend = "bitblas"  # bitblas or gemlite for 2-bit runtime
compute_dtype = torch.bfloat16 if backend == "torchao_int4" else torch.float16
cache_dir = '.'

# Device Selection: CPU
device = 'cpu'
print("GPU is not available. Using CPU.")

# Clear the Hugging Face cache
shutil.rmtree(os.path.expanduser("~/.cache/huggingface/transformers"), ignore_errors=True)

# Load the model
###################################################
model_id = 'mobiuslabsgmbh/Llama-3.2-3B-Instruct_4bitgs64_hqq_hf'

# Authenticate with Hugging Face Hub
#huggingface_hub.login()

model = None  # Initialize model to None to avoid NameError if loading fails
tokenizer = None  # Initialize tokenizer to None to avoid NameError if loading fails

try:
    # Using from_quantized to load the pre-quantized model
    model = AutoHQQHFModel.from_quantized(model_id, cache_dir=cache_dir, compute_dtype=compute_dtype, device=device, adapter='adapter_v0.1.lora').eval()
    tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
except Exception as e:
    print(f"Error loading model: {e}")
    # If initial loading fails, try manual download
    # (Replace /path/to/downloaded/model with actual path)
    local_model_path = '/path/to/downloaded/model'
    try:
        model = AutoHQQHFModel.from_pretrained(local_model_path, compute_dtype=compute_dtype, device=device, adapter='adapter_v0.1.lora').eval()
        tokenizer = AutoTokenizer.from_pretrained(local_model_path)
    except Exception as e2:
        print(f"Error loading model from local path: {e2}")
        exit()

# Use optimized inference kernels
###################################################
try:
    prepare_for_inference(model, backend=backend)  # It takes a while...
    print(f"Model prepared for inference with backend '{backend}' on {device}.")
except Exception as e:
    print(f"Error preparing model for inference: {e}")
    print("Inference might still work, but performance could be suboptimal.")

# Generate
###################################################
# For longer context, make sure to allocate enough cache via the cache_size= parameter
# gen = HFGenerator(model, tokenizer, max_new_tokens=1000, do_sample=True, compile=None) #Slower generation but no warm-up
# Check if model and tokenizer are loaded before proceeding
if model is not None and tokenizer is not None:
    gen = HFGenerator(model, tokenizer, max_new_tokens=10, do_sample=True, compile="partial").warmup()  # Faster generation, but warm-up takes a while

    print("\nGenerating text:")
    try:
        gen.generate("Write an essay about large language models", print_tokens=True)
        # gen.generate("Tell me a funny joke!", print_tokens=True)
        # gen.generate("How to make a yummy chocolate cake?", print_tokens=True)
    except Exception as e:
        print(f"Error during text generation: {e}")
        print("Please check the error message and your environment.")
else:
    print("Model or tokenizer loading failed. Please check the previous error messages.")

GPU is not available. Using CPU.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

Error loading model: Weight file missing. Check your cache directory.
Error loading model from local path: type object 'AutoHQQHFModel' has no attribute 'from_pretrained'
Error preparing model for inference: 'NoneType' object has no attribute 'named_children'
Inference might still work, but performance could be suboptimal.
Model or tokenizer loading failed. Please check the previous error messages.
